<a href="https://colab.research.google.com/github/anurag-chiplunkar/UBS_POC/blob/master/llm_email_chat_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip list > requirements.txt

In [3]:
import os
from groq import Groq
import json


In [4]:
client = Groq(
    api_key="Your_API_Key_here"
)

In [ ]:
llm = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content": """
              You are simulating a real customer writing to a support or help desk across various industries.

              Generate a highly realistic and natural customer support email. Please make each sample **different in tone, length, structure, and level of detail**. Avoid repeating the same phrases or formats.

              The email should include:
              1. A **subject** line appropriate to the issue or intent.
              2. A **body** with a natural human tone that may vary between:
                - Polite and formal
                - Conversational and informal
                - Urgent or emotional

              Randomly pick one of the following **email types**:
              - Feedback
              - Complaint
              - Request
              - Suggestion
              - Approval/confirmation
              - Escalation/follow-up

              And one of the following **domains**:
              - Finance
              - Telecom
              - E-commerce
              - Travel
              - Education
              - Healthcare

              The content should:
              - Vary between long and short emails
              - Include different fictional names, references (order ID, account number, booking ID, etc.)
              - Mention attachments or prior emails when appropriate
              - Not follow the same exact structure every time

              Return the result in this format:

              {
                "subject": "<Subject>",
                "email_body": "<Body>"
              }
              """
        }
    ],
    model="llama-3.3-70b-versatile"
    # temperature=1
)

# Sample Email Generator

In [ ]:
import random


departments = ["feedback", "complaint", "request", "suggestion", "approval", "escalation"]

# chosen_category = random.choice(categories)
# chosen_domain = random.choice(domains)

def sampleEmailGenerator():
  while True:
    llm = client.chat.completions.create(
        messages=[
            {
                "role":"system",
                "content": f"""
                  You are simulating a real customer writing to a support or help desk across various industries.\n\n

                  Generate a highly realistic and natural customer support email.\n Please make a sample **different in tone, length, structure, and level of detail**.\n Avoid repeating the same phrases or formats.\n\n

                  The email should include:\n
                  1. A **subject** line appropriate to the issue or intent.n\
                  2. A **body** with a natural human tone that may vary between:\n
                    - Polite and formal\n
                    - Conversational and informal\n
                    - Urgent or emotional\n\n

                  Pick the following **email type** as {random.choice(departments)}\n\n

                  And the following **domain** as Banking.\n\n

                  The content should:\n
                  - Vary between long and short email\n
                  - Include different fictional names, references (order ID, account number, booking ID, etc.)\n
                  - Mention attachments or prior emails when appropriate\n
                  - Not follow the same exact structure every time\n\n

                  You have to strictly follow this guideline: Generate only one sample email at a time as this prompt will be part of a generator function. Hence expecting only one sample email at a time.\n\n

                  Strictly make sure you dont add any \ or \n or newlines in the result json.

                  Return the result in json format only as example given below:

                    {{"subject": "Unresolved Issue with Account Number: 1234567890","body": "Dear Banking Support Team, I am writing to follow up on my previous email regarding the discrepancy in my account statement. Unfortunately, I have yet to receive a resolution or a satisfactory explanation for the missing funds. My account number is 1234567890, and I have attached a copy of the statement in question for your reference. As you can see from the attachment, there is a transaction dated 02/15/2023 for $500.00 that I did not authorize. I have tried to contact the bank's customer service multiple times, but the issue remains unresolved. I would greatly appreciate it if you could look into this matter urgently and provide me with a detailed explanation of the transaction and the steps being taken to rectify the situation. Please find the attachment: Account_Statement_1234567890.pdf My previous email was dated 02/20/2023, with the subject 'Discrepancy in Account Statement'. I have also included my contact information below for your convenience. Thank you for your prompt attention to this matter. Sincerely, Emily J. Lee Account Holder"}}

                  """
            }
        ],
        model="llama-3.3-70b-versatile",
        temperature=0.9      # adds randomness

        # top_k=40,            # limits candidates to top k tokens
        # top_p=0.9,           # uses nucleus sampling for diversity
        # repeat_penalty=1.1   # discourages repeating tokens
    )
    sampleEmail = llm.choices[0].message.content
    yield sampleEmail

In [ ]:
emailGenerator = sampleEmailGenerator()

In [ ]:
print(next(emailGenerator))

{"subject": "Overdraft Fees on Account 9876543210","body": "Hi Banking Support, I'm reaching out about the overdraft fees I was charged on my account 9876543210. I checked my account online and saw that I was charged $35 for overdraft on 3 separate transactions. I understand that I went below the minimum balance, but I had set up overdraft protection from my savings account. I've attached a screenshot of my account activity for your reference. Could you please look into this and refund the fees? I've been a customer for over 5 years and have never had this issue before. My previous email about this issue was on 03/10/2023, but I haven't received a response yet. Please let me know if there's any additional information you need from me. Thanks, David Kim"}


## Creating empty list

In [ ]:
listOfJsonEmails = []

In [ ]:
for _ in range(1000):
  emailSample = next(emailGenerator)
  listOfJsonEmails.append(json.loads(emailSample))

## Saving to JSON file

In [ ]:
listOfJsonEmails

In [ ]:
with open("uniqueJsonEmails-new.json", "w") as file:
  json.dump(listOfJsonEmails, file, indent=4)

# Sample Chat Generator

In [5]:
import random

categories = ["feedback", "complaint", "request", "suggestion", "approval", "escalation"]


def chatMessageGenerator():
  while True:
    llm = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"""
                  You are simulating a customer service **chat conversation** between a customer and a support desk agent.\n\n

                  Your goal is to randomly generate a realistic chat exchange, consisting of **5 to 10 back-and-forth messages** (total), relevant to:\n\n

                  - **Category as**: {random.choice(categories)}n\
                  - **Domain as** Banking\n\n

                  💬 Guidelines:\n
                  - The conversation should feel real and natural — not robotic or templated.\n
                  - Vary the message tone, structure, and length (e.g., short queries, detailed explanations).\n
                  - Include realistic references like account numbers, order IDs, ticket references, etc.\n
                  - Mention previous communication or uploaded documents where appropriate.\n
                  - Agent responses should be professional, empathetic, and accurate for the scenario.\n\n

                  📦 Output format:\n
                  Return the chat as **a valid JSON array of objects**, where each object follows this structure:\n\n

                  [{{"sender": "customer","message": "Hi, I’m reaching out because I was double-charged for my last order..."}},{{"sender": "agent","message": "I’m really sorry to hear that. Let me quickly check the billing details for you."}},...]

                  ⚠️ Output ONLY the JSON list — no explanation, notes, or extra formatting.
                  """
                          }
                      ],
                      model="llama-3.3-70b-versatile",
                      temperature=0.9      # adds randomness
                  )

    sampleChat = llm.choices[0].message.content
    yield sampleChat


In [6]:
chatGenerator = chatMessageGenerator()
print(next(chatGenerator))

[
  {"sender": "customer", "message": "Hi, I'm trying to request a refund for an incorrect transaction on my account #1234567890"},
  {"sender": "agent", "message": "I'd be happy to help you with that. Can you please provide me with the transaction date and amount so I can look into this further?"},
  {"sender": "customer", "message": "The transaction date was 2023-02-15 and the amount was $500. I've already sent an email with the details to your support team, reference #REF20230215"},
  {"sender": "agent", "message": "Thank you for providing that information. I've located your email and the transaction in question. I'm going to go ahead and process a refund for the incorrect amount. You should see the credit back in your account within 3-5 business days."},
  {"sender": "customer", "message": "That sounds great, thank you so much for your help! Can you also let me know if there's anything else I need to do on my end?"},
  {"sender": "agent", "message": "No, that's all taken care of. H

In [7]:
listOfJsonChats = []

In [8]:
for i in range(2):
  listOfJsonChats.append((json.loads(next(chatGenerator))))

In [9]:
with open("uniqueJsonChat.json", "w") as file:
  json.dump(listOfJsonChats, file, indent=4)

# Voice Sample Generator

In [ ]:
import random

categories = ["feedback", "complaint", "request", "suggestion", "approval", "escalation"]


def voiceScriptGenerator():
  while True:
    llm = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"""
                Generate a realistic customer voice message to a support desk based on the category {random.choice(categories)} and domain banking. The tone should sound like it’s being spoken, not written. Use filler words, pauses, informal phrasing. Make sure it is atleast 15 sentence long.

                Output should be in JSON:
                {{"transcript": "<spoken text>"}}

                """
            }
        ],
        model="llama-3.3-70b-versatile",
        temperature=0.9      # adds randomness
        )

    sampleChat = llm.choices[0].message.content
    yield sampleChat


In [ ]:
voiceScript = voiceScriptGenerator()
print(next(voiceScript))

{"transcript": "Hey, um, hi, I'm calling about my account, right, so I've been trying to, you know, log in online, and it's not working, like, at all. I've tried, uh, resetting my password, and, yeah, that didn't work either. I've been on the website for, like, 20 minutes now, and I'm getting really frustrated, you know? So, I was wondering if, maybe, you guys could help me out. I've got, like, a few questions, and I'm not really sure what's going on. Okay, so, first of all, my account number is, uh, 123456789, and, yeah, I've had it for, like, five years now. I've never had any problems before, but, like, now it's just not working. I've tried, you know, calling the, uh, the phone number on the back of my card, but, yeah, it just keeps sending me to voicemail. So, I'm like, really stuck here, and I don't know what to do. I need to, you know, pay some bills, and I'm starting to get, like, really worried that I'm gonna be late, you know? Can you, like, help me figure out what's going on?

In [ ]:
listOfJsonVoiceScripts = []

In [ ]:
for i in range(2):
  listOfJsonVoiceScripts.append((json.loads(next(voiceScript))))

In [ ]:
with open("uniqueJsonVoiceScripts.json", "w") as file:
  json.dump(listOfJsonVoiceScripts, file, indent=4)